In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from datetime import datetime
from datetime import date

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error

In [17]:
full_df = pd.read_csv('/content/drive/MyDrive/Omdena_Dengue_fever_Sri_Lanka/Sri Lanka Dengue models/Copy of Sri_lanka_dengue_cases_weather_weekly_2007_2024_ updated.csv')

In [18]:
full_df.head()

,District,Number_of_Cases,Week_Start_Date,Month,Year,Week,Week_End_Date,Avg Max Temp (°C),Avg Min Temp (°C),Avg Apparent Max Temp (°C),Avg Apparent Min Temp (°C),Total Precipitation (mm),Total Rain (mm),Avg Wind Speed (km/h),Max Wind Gusts (km/h),Weather Code,Avg Daylight Duration (hours),Avg Sunrise Time,Avg Sunset Time
0,Ampara,0,2007-01-01,1,2007,1,2007-01-08,26.9375,21.8625,28.8500,24.1000,8.0,8.0,18.2000,44.3,3,11.714583,376,1079
1,Ampara,0,2007-01-08,1,2007,2,2007-01-15,26.3500,23.3250,30.2625,26.7000,86.1,86.1,17.0250,43.6,53,11.733333,378,1082
2,Ampara,0,2007-01-15,1,2007,3,2007-01-22,27.2125,23.1625,29.5750,25.9625,3.1,3.1,20.1875,48.2,51,11.754167,380,1085
3,Ampara,0,2007-01-22,1,2007,4,2007-01-29,26.7250,23.0375,29.9500,26.1125,53.6,53.6,17.2000,42.5,61,11.783333,381,1088
4,Ampara,0,2007-01-29,1,2007,5,2007-02-05,27.3625,22.9625,30.7500,26.1250,8.2,8.2,17.2625,40.0,51,11.812500,382,1090


In [19]:
districts = full_df['District'].unique()
districts

array(['Ampara', 'Anuradhapura', 'Badulla', 'Batticaloa', 'Colombo',
       'Galle', 'Gampaha', 'Hambantota', 'Jaffna', 'Kalutara', 'Kandy',
       'Kegalle', 'Kilinochchi', 'Kurunegala', 'Mannar', 'Matale',
       'Matara', 'Monaragala', 'Mullaitivu', 'NuwaraEliya', 'Polonnaruwa',
       'Puttalam', 'Ratnapura', 'Trincomalee', 'Vavuniya'], dtype=object)

In [20]:
cols = ['District','Number_of_Cases', 'Week_Start_Date', 'Month', 'Avg Min Temp (°C)', 'Total Precipitation (mm)',
        'Avg Wind Speed (km/h)', 'Avg Daylight Duration (hours)']

In [21]:
df = full_df[cols]
df.head()

,District,Number_of_Cases,Week_Start_Date,Month,Avg Min Temp (°C),Total Precipitation (mm),Avg Wind Speed (km/h),Avg Daylight Duration (hours)
0,Ampara,0,2007-01-01,1,21.8625,8.0,18.2000,11.714583
1,Ampara,0,2007-01-08,1,23.3250,86.1,17.0250,11.733333
2,Ampara,0,2007-01-15,1,23.1625,3.1,20.1875,11.754167
3,Ampara,0,2007-01-22,1,23.0375,53.6,17.2000,11.783333
4,Ampara,0,2007-01-29,1,22.9625,8.2,17.2625,11.812500


In [45]:
Ntest = 12
T = 12

In [46]:
for district in districts:
  print(len(df[df['District'] == district]))

916
916
916
916
916
916
916
916
916
916
916
916
916
916
916
916
916
916
916
916
916
916
916
916
916


In [47]:
L = 916

## Random Forest Regressor

In [59]:
def apply_rfr(district):
  district_df = df[df['District']==district]
  district_df.reset_index(drop=True, inplace=True)
  district_df['Week_Start_Date'] = pd.to_datetime(district_df['Week_Start_Date'])
  district_df = district_df.sort_values(by='Week_Start_Date')
  district_df.set_index('Week_Start_Date', inplace=True)
  district_df.drop(columns=['District'], inplace=True)
  Tx = T
  Ty = Ntest
  X = []
  Y = []
  for t in range(L - Tx - Ty + 1):
    x = district_df[t:t+Tx].to_numpy().flatten()
    X.append(x)
    y = district_df['Number_of_Cases'][t+Tx:t+Tx+Ty]
    Y.append(y)

  Xtrain, Ytrain = X[:-1], Y[:-1]
  Xtest, Ytest = X[-1:], Y[-1:]

  rfr = RandomForestRegressor(random_state=42)
  rfr.fit(Xtrain, Ytrain)

  rmse = np.sqrt(mean_squared_error(rfr.predict(Xtest[0].reshape(1,-1)).flatten(), Ytest[0]))
  print(f"{district} rmse:", rmse)
  return rmse

In [61]:
RFR_RMSE = []
for district in districts:
  rmse = apply_rfr(district)
  RFR_RMSE.append(rmse)

Ampara rmse: 5.094391524019331
Anuradhapura rmse: 3.49461252024694
Badulla rmse: 8.710527155880598
Batticaloa rmse: 6.075983459490324
Colombo rmse: 22.4130417763111
Galle rmse: 15.645886040745664
Gampaha rmse: 20.553866019153997
Hambantota rmse: 11.655046474953814
Jaffna rmse: 12.022929135614167
Kalutara rmse: 12.72771909390419
Kandy rmse: 54.97840068305613
Kegalle rmse: 27.583578599110982
Kilinochchi rmse: 4.451888176193707
Kurunegala rmse: 14.534735693044668
Mannar rmse: 3.625742452703078
Matale rmse: 5.196161243328259
Matara rmse: 12.323200680018159
Monaragala rmse: 4.057570085654714
Mullaitivu rmse: 1.2647265844178863
NuwaraEliya rmse: 3.3292241338385935
Polonnaruwa rmse: 2.8072658703205624
Puttalam rmse: 6.672370892968505
Ratnapura rmse: 74.079927949479
Trincomalee rmse: 4.061863283436966
Vavuniya rmse: 1.0328641407916792


In [63]:
np.array(RFR_RMSE).mean()

13.535740946747321

## XGBoost

In [64]:
!pip install xgboost

In [65]:
from xgboost import XGBRegressor

In [66]:
def apply_xgb(district):
  district_df = df[df['District']==district]
  district_df.reset_index(drop=True, inplace=True)
  district_df['Week_Start_Date'] = pd.to_datetime(district_df['Week_Start_Date'])
  district_df = district_df.sort_values(by='Week_Start_Date')
  district_df.set_index('Week_Start_Date', inplace=True)
  district_df.drop(columns=['District'], inplace=True)
  Tx = T
  Ty = Ntest
  X = []
  Y = []
  for t in range(L - Tx - Ty + 1):
    x = district_df[t:t+Tx].to_numpy().flatten()
    X.append(x)
    y = district_df['Number_of_Cases'][t+Tx:t+Tx+Ty]
    Y.append(y)

  Xtrain, Ytrain = X[:-1], Y[:-1]
  Xtest, Ytest = X[-1:], Y[-1:]

  xgb = XGBRegressor(random_state=42)
  xgb.fit(Xtrain, Ytrain)

  rmse = np.sqrt(mean_squared_error(xgb.predict(Xtest[0].reshape(1,-1)).flatten(), Ytest[0]))
  print(f"{district} rmse:", rmse)
  return rmse

In [67]:
XGB_RMSE = []
for district in districts:
  rmse = apply_xgb(district)
  XGB_RMSE.append(rmse)

Ampara rmse: 7.004327448966989
Anuradhapura rmse: 4.1241059698347415
Badulla rmse: 7.763754541096746
Batticaloa rmse: 7.569777408683403
Colombo rmse: 44.78472639779911
Galle rmse: 27.75299944787548
Gampaha rmse: 45.19080767824294
Hambantota rmse: 17.05604414033456
Jaffna rmse: 19.373446132506608
Kalutara rmse: 27.77556963677283
Kandy rmse: 88.96915447214181
Kegalle rmse: 35.60817433561384
Kilinochchi rmse: 3.157790498060744
Kurunegala rmse: 22.515876401980105
Mannar rmse: 3.8473370566157756
Matale rmse: 13.297792478246103
Matara rmse: 14.671118965003847
Monaragala rmse: 7.243118351217508
Mullaitivu rmse: 1.2219653029365491
NuwaraEliya rmse: 2.585897439692023
Polonnaruwa rmse: 3.438476189422744
Puttalam rmse: 5.476035089656895
Ratnapura rmse: 124.7683163584577
Trincomalee rmse: 4.933638256477604
Vavuniya rmse: 1.4971396613554278


In [68]:
np.array(XGB_RMSE).mean()

21.66509558635968